In [1]:
%matplotlib qt

import time

import matplotlib.pyplot as plt

import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

# PyExaFMM
from fmm import Fmm
from fmm.kernel import laplace_p2p_serial
from fmm.surface import scale_surface

## Generate Test Data
Uncomment to generate test data

In [2]:
# ! fmm generate-test-data -c C2E2 && fmm compute-operators -c C2E2
# ! fmm generate-test-data -c C3E3 && fmm compute-operators -c C3E3
# ! fmm generate-test-data -c C4E4 && fmm compute-operators -c C4E4
# ! fmm generate-test-data -c C5E5 && fmm compute-operators -c C5E5
# ! fmm generate-test-data -c C6E6 && fmm compute-operators -c C6E6
# ! fmm generate-test-data -c C7E7 && fmm compute-operators -c C7E7

## Compare Analyticity of Expansions from PyExaFMM and ExaFMM-T

How does discretisation effect the quality of the expansions? We check the analyticity of multipole expansions by computing the approximated potential at points distributed on the surface of a sphere surrounding the equivalent surface, in the far-field. As the radius of this sphere is increased, we can demonstrate the accuracy of the multipole expansion away from the equivalent surface. We repeat a similar experiment for the local expansions.

In [3]:
def sphere_data(npoints, r, c):
    """
    Generate npoints randomly distributed on the
        surface of a sphere with radius r, and center c
    """
    np.random.seed(0)
    
    phi = np.random.rand(npoints)*2*np.pi
    costheta = (np.random.rand(npoints)-0.5)*2

    theta = np.arccos(costheta)

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    sources = np.vstack((x, y, z))

    return sources.T + c


In [4]:
test = sphere_data(1000, 10, np.array([10, 10, 0]))

ax = plt.axes(projection='3d')
ax.scatter3D(test[:, 0], test[:, 1], test[:, 2], c='k', s=1)
plt.show()

In [5]:
# Discretisation order of check surface
Cvec = [2, 3, 4, 5, 6, 7]

# Discretisation order of equivalent surface
Evec = [2, 3, 4, 5, 6, 7]

# Load experimental data into PyExaFMM
pyfmmvec = [Fmm(f'C{C}E{C}') for C in Cvec]

In [6]:
# Load experimental data into ExaFMM-T
exafmmvec = []
exafmmtreevec = []

for e in pyfmmvec:
    # create a list of source instances
    sources = laplace.init_sources(e.sources, e.source_densities)

    # create a list of target instances
    targets = laplace.init_targets(e.targets)
    
    # Expansion order
    p = e.config['order_equivalent']
    fmm = laplace.LaplaceFmm(p=p, ncrit=e.config['max_points'], filename=f'C{p}E{p}.dat')
    exafmmvec.append(fmm)
    
    tree = laplace.setup(sources, targets, fmm)
    exafmmtreevec.append(tree)

In [7]:
# Evaluate PyExaFMM experiments
for e in pyfmmvec:
    e.run()

In [8]:
# Evaluate ExaFMM-T experiments
ex_target_potentials = []
for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    ex_target_potentials.append(laplace.evaluate(tree, fmm))

In [9]:
# Create surfaces

equivalent_surfaces = [
    e.equivalent_surface for e in pyfmmvec
]

upward_equivalent_surfaces = [scale_surface(
    surf=equivalent_surfaces[i], radius=pyfmmvec[i].r0, 
    level=0, center=pyfmmvec[i].x0, alpha=pyfmmvec[i].alpha_outer
) for i in range(len(pyfmmvec))]

# Extract multipole expansion at root node
pyfmm_multipole_expansions = [
    e.multipole_expansions[e.key_to_index[0]:e.key_to_index[0]+e.nequivalent_points]
    for e in pyfmmvec
]

exafmm_multipole_expansions = [
    np.array(tree.nodes[0].up_equiv) for tree in exafmmtreevec
]

In [20]:
# Create spheres centered on the root node with radii such that
# the root node's multipole expansion is valid
npoints = 1000
r0 = pyfmmvec[0].r0
x0 = pyfmmvec[0].x0

# Start of far-field
r = r0*3
c = x0
rvec = np.linspace(r, 10*r, 10)
spherevec = [sphere_data(npoints, r, c) for r in rvec]

# Experiment index with a given discretisation
e_idx = 0

# Store results for each experiment
pyfmm_results = [[] for i in range(len(pyfmmvec))]
exafmm_results = [[] for i in range(len(pyfmmvec))]
direct_results = [[] for i in range(len(pyfmmvec))]


# Compare expansion results with direct computation at spheres of increasing radius
# surrounding the upward equivalent surface of the root node.
for e_idx in range(len(pyfmmvec)):
    for i, sphere in enumerate(spherevec):

        pyfmm_results[e_idx].append(
            laplace_p2p_serial(
                sources=upward_equivalent_surfaces[e_idx],
                targets=sphere,
                source_densities=pyfmm_multipole_expansions[e_idx]
            )
        )

        exafmm_results[e_idx].append(
            laplace_p2p_serial(
                sources=upward_equivalent_surfaces[e_idx],
                targets=sphere,
                source_densities=exafmm_multipole_expansions[e_idx]
            )
        )

        direct_results[e_idx].append(
            laplace_p2p_serial(
                sources=pyfmmvec[e_idx].sources,
                targets=sphere,
                source_densities=pyfmmvec[e_idx].source_densities
            )
        )
        
pyfmm_results = np.array(pyfmm_results)
exafmm_results = np.array(exafmm_results)
direct_results = np.array(direct_results)

In [38]:
# Compute relative errors of multipole expansions of PyFMM and ExaFMM-T
pyfmm_relative_errors = [[] for i in range(len(pyfmmvec))]
exafmm_relative_errors = [[] for i in range(len(pyfmmvec))]

for e_idx in range(len(pyfmmvec)):
    for sph_idx in range(len(spherevec)):
        direct = direct_results[e_idx][sph_idx]
        pyfmm_estimate = pyfmm_results[e_idx][sph_idx]
        exafmm_estimate = exafmm_results[e_idx][sph_idx]
        
        pyfmm_err = abs(direct-pyfmm_estimate)/direct
        exafmm_err = abs(direct-exafmm_estimate)/direct

        pyfmm_relative_errors[e_idx].append(np.mean(pyfmm_err))
        exafmm_relative_errors[e_idx].append(np.mean(exafmm_err))
        
pyfmm_relative_errors = np.array(pyfmm_relative_errors)
exafmm_relative_errors = np.array(exafmm_relative_errors)

In [65]:
fig = plt.figure()
ax = plt.subplot(1, 1, 1)

lines = ['solid', 'dashed', 'dashdot', 'dotted', 'solid', 'dashed']
markers = ['.', 'o', 'v', '+', 'D', 's']

for e_idx in range(len(pyfmmvec)):
    ax.semilogy(rvec, pyfmm_relative_errors[e_idx], marker=markers[e_idx], linestyle=lines[e_idx])

# Label with number of coefficients
legend_labels = [ 6*(order-1)**2 + 2 for order in Evec]
plt.legend(legend_labels)
plt.show()

In [66]:
fig = plt.figure()
ax = plt.subplot(1, 1, 1)

lines = ['solid', 'dashed', 'dashdot', 'dotted', 'solid', 'dashed']
markers = ['.', 'o', 'v', '+', 'D', 's']

for e_idx in range(len(pyfmmvec)):
    ax.semilogy(rvec, exafmm_relative_errors[e_idx], marker=markers[e_idx], linestyle=lines[e_idx])

# Label with number of coefficients
legend_labels = [ 6*(order-1)**2 + 2 for order in Evec]
plt.legend(legend_labels)
plt.show()

In [37]:
pyfmm_relative_errors[0][0]

array([0.42867854, 0.43511674, 0.4199581 , 0.41926578, 0.42545366,
       0.4014753 , 0.4285997 , 0.42876205, 0.41624534, 0.4238743 ,
       0.43118846, 0.41798764, 0.40998587, 0.424855  , 0.43199247,
       0.41864854, 0.43663183, 0.39907503, 0.4335085 , 0.38846952,
       0.42069635, 0.4266538 , 0.42772707, 0.42592746, 0.3870016 ,
       0.3935697 , 0.41966975, 0.40811798, 0.4370109 , 0.4030692 ,
       0.43704617, 0.42313036, 0.42428416, 0.42837748, 0.42484003,
       0.4049282 , 0.39035606, 0.39099923, 0.4108753 , 0.42718804,
       0.39565077, 0.41557667, 0.4252061 , 0.4114864 , 0.43093497,
       0.42230892, 0.42637044, 0.39361018, 0.4277756 , 0.42892465,
       0.4065546 , 0.4341006 , 0.437315  , 0.39298356, 0.41298738,
       0.39562303, 0.4023077 , 0.42092732, 0.42208776, 0.43684456,
       0.4016868 , 0.41508016, 0.4363491 , 0.43458357, 0.43078417,
       0.41142747, 0.42381167, 0.42797992, 0.4003408 , 0.39635897,
       0.43765387, 0.42451116, 0.42385972, 0.39008465, 0.42843